In [ ]:
%load_ext autoreload
%autoreload 2
from datamodel.Track import Track
from datamodel.User import User
from mongodb import MongoAccess as ma
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from pprint import pprint

# MongoAccess.load_kaggle_csvs_into_mongodb()

df_genres = pd.DataFrame(ma.get_collection(ma.coll_genres))
df_years = pd.DataFrame(ma.get_collection(ma.coll_years))
df_tracks = pd.DataFrame(ma.get_collection(ma.coll_tracks))
df_artists = pd.DataFrame(ma.get_collection(ma.coll_artists))

a_track_id = '3jW3k9jZRHNZfpDxpbAsbn'

def retrieve_track_attr_dict(id=None):
    if not id: row = df_tracks.sample(1).iloc[0]
    else: row = df_tracks.loc[df_tracks['_id'] == id].iloc[0]  # removed duplicates so there is only 1 match
    return row.to_dict()

In [ ]:
a_user = User()
print(a_user.get_optimal_track_pref())
a_user.update_track_attr_log(Track(retrieve_track_attr_dict()))
print(a_user.get_optimal_track_pref())
a_user.update_track_attr_log(Track(retrieve_track_attr_dict()))
print(a_user.get_optimal_track_pref())
a_user.update_track_attr_log(Track(retrieve_track_attr_dict()))
print(a_user.get_optimal_track_pref())
df_tracks.head()

In [ ]:
# can use lists of names/ids
pprint(ma.get_tracks_by_ids(['0hbkKFIJm7Z05H8Zl9w30f', a_track_id]))
# or single name/id
pprint(ma.get_tracks_by_names("Don't You Advertise Your Man"))

In [ ]:
pipeline = [
    { '$limit' : 10},
]
res = list(ma.coll_tracks.aggregate(list(pipeline)))
pprint(res)

In [ ]:
import dotenv
# i'm using a .env file with the following information for the following part to work, not sure if you need all this info but I had it from another spotify project I am working on 
# SPOTIPY_CLIENT_ID=xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
# SPOTIPY_CLIENT_SECRET=yyyyyyyyyyyyyyyyyyyyyyyyyyyyyyy
# SPOTIPY_REDIRECT_URI=http://sdfsdfsdfsd:8080
# https://developer.spotify.com/documentation/general/guides/app-settings/
dotenv.load_dotenv()

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials, SpotifyOAuth

sp = spotipy.Spotify(auth_manager=SpotifyOAuth())


In [ ]:
# not done yet will be used to be able to get all tracks from a certain genre

album = sp.album(sp.track('spotify:track:6BL4PpswVjH9BAajbIbpmZ')['album']['uri'])
genres = []
if len(album['genres']) == 0: # spotify doesn't actually have genres on track/album level https://github.com/spotify/web-api/issues/157
    for artist in album['artists']:
        for genre in sp.artist(artist['uri'])['genres']:
            genres.append(genre)
else:
    for genre in album['genres']:
        genres.append(genre)

genre_set = set(genres)
genre_set

In [ ]:
# creates the album collection

# def get_album_data(album_id):
#     album = sp.album(album_id)
#     results = album['tracks'] 
#     while results['next']: # if doc right would only happen if album contains more than a 100 songs, which is never according to my googling
#         results = sp.next(results)
#         album['tracks']['items'].extend(results['items'])
        
#     # there is some info in the album that we do not need as we have seperate track and artist collections
#     album_bare = dict()
#     for key in ['id', 'album_type', 'genres', 'label', 'name', 'popularity', 'release_date', 'release_date_precision', 'total_tracks', 'type']:
#         album_bare[key] = album[key]
#     album_bare['artists'] = [{'id':artist['id']} for artist in album['artists']]
#     album_bare['tracks'] = [{'id':track['id']} for track in album['tracks']['items']]
#     return album_bare

# album_list = []
# for track in all_tracks:
#     album_id = sp.track(track['id'])['album']['id']
#     ma.coll_tracks.update_one({'_id': track['_id']}, {'$set': {'album_id': album_id}})
#     album_list.append(album_id)

# album_data_list = [get_album_data(album_id) for album_id in set(album_list)]
# ma.coll_albums.insert_many(album_data_list)